In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
df = pd.read_csv('titled_players_raw.csv',index_col=0)

In [3]:
df.head()

,IDcode,Name,T,WT,Oth. T.,Fed,Rtg,Rpd,Blz,B-Year,S,F,Title
0,5813298,"Lim, Kok Ann",CM,,,SGP,,,,1920,M,,"['Candidate Master (CM)', '2012']"
1,4611870,"Keller-Hermann, Edith",WGM,WGM,,GER,2290,,,1921,F,i,"['Woman Grandmaster (WGM)', '1978', 'Woman Int..."
2,2002310,"Levy, Louis",FM,,,USA,2275,,,1921,M,i,[]
3,14100657,"Ousatchi, Mark",FM,,,GER,2157,,,1921,M,i,[]
4,4101529,"Averbakh, Yuri L",GM,,,RUS,2445,,,1922,M,i,"['Grandmaster (GM)', '1952', 'International Ma..."


In [4]:
df = df.replace({' ':np.nan})

In [5]:
df.head()

,IDcode,Name,T,WT,Oth. T.,Fed,Rtg,Rpd,Blz,B-Year,S,F,Title
0,5813298,"Lim, Kok Ann",CM,NaN,NaN,SGP,NaN,NaN,NaN,1920,M,NaN,"['Candidate Master (CM)', '2012']"
1,4611870,"Keller-Hermann, Edith",WGM,WGM,NaN,GER,2290,NaN,NaN,1921,F,i,"['Woman Grandmaster (WGM)', '1978', 'Woman Int..."
2,2002310,"Levy, Louis",FM,NaN,NaN,USA,2275,NaN,NaN,1921,M,i,[]
3,14100657,"Ousatchi, Mark",FM,NaN,NaN,GER,2157,NaN,NaN,1921,M,i,[]
4,4101529,"Averbakh, Yuri L",GM,NaN,NaN,RUS,2445,NaN,NaN,1922,M,i,"['Grandmaster (GM)', '1952', 'International Ma..."


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19510 entries, 0 to 19509
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   IDcode   19510 non-null  int64 
 1   Name     19510 non-null  object
 2   T        19510 non-null  object
 3   WT       3929 non-null   object
 4   Oth. T.  105 non-null    object
 5   Fed      19510 non-null  object
 6   Rtg      19436 non-null  object
 7   Rpd      11811 non-null  object
 8   Blz      12074 non-null  object
 9   B-Year   19508 non-null  object
 10  S        19510 non-null  object
 11  F        8260 non-null   object
 12  Title    19510 non-null  object
dtypes: int64(1), object(12)
memory usage: 2.1+ MB


La colonne Oth. T. est vide à 99% et nous donne des informations sur les joueurs ayant reçu des titres pour l'arbritrage. Cela ne nous intéresse pas.

Eliminons la

In [7]:
df = df.drop('Oth. T.', axis=1)

La colonne WT est vide à 79%

In [8]:
df['WT'].value_counts(normalize=True, dropna=False)

NaN     0.798616
 WFM    0.091748
 WIM    0.043875
 WCM    0.041312
 WGM    0.024398
 WH     0.000051
Name: WT, dtype: float64

Néanmoins 79% des titres sont détenus par les hommes

In [9]:
df['S'].value_counts(normalize=True)

M    0.797335
F    0.202665
Name: S, dtype: float64

Enlevons les espaces inutiles dans les colonnes de types object

In [10]:
cols = df.select_dtypes(include=['object']).columns
for col in cols:
    df[col] = df[col].str.strip()

In [11]:
df = df.rename(columns = {'IDcode':'id', 
           'Name':'nom',
           'T':'title',
           'WT':'woman_titre',
           'Fed':'pays',
           'Rtg':'classique',
           'Rpd':'rapide',
           'Blz':'blitz',
           'B-Year':'naissance',
           'S':'sexe',
           'F':'actif',
           'Title':'titre_dates'
          })

In [12]:
df[['classique', 'rapide', 'blitz', 'naissance']] = df[['classique', 'rapide', 'blitz', 'naissance']].astype('float')

Remplaçons la colonne actif par 1 si le joueur est actif 0 sinon

In [13]:
df['actif'].value_counts()

i    8260
Name: actif, dtype: int64

A vérifier si l'in/activité des joueurs semble correct

In [14]:
df['actif'] = df['actif'].replace({'i':'0',np.nan:'1'})


Il faut réfléchir à comment remplacer les obtentions de titres de chaque joueur 

In [15]:
df.iloc[df['title'].apply(lambda x:len(x)).sort_values(ascending=False)[:100].index].head()

,id,nom,title,woman_titre,pays,classique,rapide,blitz,naissance,sexe,actif,titre_dates
9755,2219824,"Redondo Arguelles, Graciela",WFM,WFM,ESP,2031.0,1984.0,NaN,1979.0,F,1,"['Woman FIDE Master (WFM)', '2004', 'Title', '..."
11931,13602470,"Tsereteli, Tamar",WGM,WGM,GEO,2306.0,NaN,2312.0,1985.0,F,0,"['Woman Grandmaster (WGM)', '2005', 'Woman Int..."
11969,14105560,"Zemlickova, Olena",WIM,WIM,CZE,2014.0,2122.0,2102.0,1985.0,F,1,"['Woman International Master (WIM)', '2015', '..."
11966,12801496,"Zaksaite, Salomeja",WIM,WIM,LTU,2202.0,2140.0,2002.0,1985.0,F,0,"['Woman International Master (WIM)', '2003']"
11956,933376,"Vranesevic, Danica",WFM,WFM,SRB,2032.0,NaN,NaN,1985.0,F,0,[]


Il semblerait que lorsqu'il y'a le mot 'Title' dans la colonne titre_dates, les informations ne sont pas utiles

In [16]:
df.head()

,id,nom,title,woman_titre,pays,classique,rapide,blitz,naissance,sexe,actif,titre_dates
0,5813298,"Lim, Kok Ann",CM,NaN,SGP,NaN,NaN,NaN,1920.0,M,1,"['Candidate Master (CM)', '2012']"
1,4611870,"Keller-Hermann, Edith",WGM,WGM,GER,2290.0,NaN,NaN,1921.0,F,0,"['Woman Grandmaster (WGM)', '1978', 'Woman Int..."
2,2002310,"Levy, Louis",FM,NaN,USA,2275.0,NaN,NaN,1921.0,M,0,[]
3,14100657,"Ousatchi, Mark",FM,NaN,GER,2157.0,NaN,NaN,1921.0,M,0,[]
4,4101529,"Averbakh, Yuri L",GM,NaN,RUS,2445.0,NaN,NaN,1922.0,M,0,"['Grandmaster (GM)', '1952', 'International Ma..."


In [17]:
# # Change la colonne titre_dates de str à list
df.loc[df['titre_dates'].dropna().index, 'titre_dates'] = df['titre_dates'].dropna().apply(lambda x: ast.literal_eval(x))

In [18]:
def remove_titre(titre_list):
    """Enlève tout les éléments de la lise si 'Title' est présent dans titre_list
    Args :
        titre_list (list): liste de la colonne df['titre_dates']
    
        Returns:
            list : liste modifié si modifications nécessaires, sinon la liste sans changement
        
    
    """
    if 'Title' in titre_list:
        return titre_list[:titre_list.index('Title')]
    return titre_list

In [19]:
df.loc[df['titre_dates'].dropna().apply(remove_titre).index, 'titre_dates'] = df['titre_dates'].dropna().apply(remove_titre)

In [20]:
df['titre_dates'] = df['titre_dates'].apply(lambda x: np.nan if (len(x)==0) else x)

Ca à l'air d'être bon

In [21]:
new_cols = [
            'Woman Candidate Master (WCM)',
            'Candidate Master (CM)',
            'Woman FIDE Master (WFM)',
            'FIDE Master (FM)',
            'Woman International Master (WIM)',
            'International Master (IM)',
            'Woman Grandmaster (WGM)',
            'Grandmaster (GM)'
           ]
dict_cols = {}
for col in new_cols:
    dict_cols[col] = np.nan
dict_default = dict_cols.copy()

In [22]:
# for titre_dates,idx in zip(df['titre_dates'].dropna(),df['titre_dates'].dropna():
#     titre_reshaped = np.reshape(titre_dates, (-1,2))
# #     print(titre_reshaped)
#     dict_cols = dict_default.copy()
#     for i in titre_reshaped:
#         title = i[0]
#         year = i[1]
#         dict_cols[title] = year


In [23]:
def fill_columns(titre_dates):
    titre_reshaped = np.reshape(titre_dates, (-1,2))
    dict_cols = dict_default.copy()
    for i in titre_reshaped:
        title = i[0]
        year = i[1]
        dict_cols[title] = year
    return dict_cols

In [24]:
dict_tofill = df['titre_dates'].dropna().apply(fill_columns)
for i in dict_tofill.index:
#     df.loc[i, dict_tofill[i].keys()] = dict_tofill.values()
    df.loc[i, dict_tofill[i].keys()] = dict_tofill[i].values()
#     print(dict_tofill[i].values())
# dict_tofill[0].keys()
# dict_tofill[0].values()